In [1]:
# imports
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

# Read Voting History

In [2]:
# read in the voting history data
votes = pd.read_excel('voting_history.xlsx')
votes.head()

,voting_date,ticker,decision,yes,no
0,2024-04-14,PL,LONG,24,6
1,2024-04-10,PGY,SHORT,13,14
2,2024-04-10,CREX,LONG,26,2
3,2024-09-10,CLBT,SHORT,21,1
4,2024-04-14,TOST,LONG,28,3


# Obtain Historical Price Data

In [3]:
# Add in 1mo, 6mo, 1yr return columns to the votes dataframe
votes_with_returns = votes.copy()
votes_with_returns['1mo_return'] = None
votes_with_returns['6mo_return'] = None
votes_with_returns['1yr_return'] = None

In [17]:
# Method for getting n-period returns of equity with yfinance
def get_n_period_return(ticker):
    ticker = yf.Ticker(ticker)
    data = ticker.history(period='max')
    return data


In [18]:
rs = get_n_period_return('TSLA')
rs

TSLA: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


# Calculate Alphas

alpha = true if N-period return of equity > N-period return of benchmark else false